In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

# Load the dataset

In [ ]:
!curl -O https://raw.githubusercontent.com/akshayjoshii/COVID19-Tweet-Sentiment-Analysis-and-EDA/master/finalSentimentdata2.csv

In [ ]:
df = pd.read_csv("finalSentimentdata2.csv")

In [ ]:
df.head()

In [ ]:
sns.countplot(x=df['sentiment']);

# Data Cleaning

In [ ]:
import nltk
import re
import string

In [ ]:
def clean_text(text):
    # Make text lowercase
    text = text.lower()
    # Remove text within square brackets
    text = re.sub('\[.*?\]', '', text)
    # Remove URLs
    text = re.sub('https?://\S+|www\.\S+', '', text)
    # Remove text within <>
    text = re.sub('<.*?>+', '', text)
    # Remove punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # Remove newline
    text = re.sub('\n', '', text)
    # Remove words containing numbers
    text = re.sub('\w*\d\w*', '', text)
    # Remove unicode emojis (todo) but this could mean something! 
    return text

In [ ]:
# Apply `clean_text` to the text element of the dataframe

df['text'] = df['text'].apply(lambda x: clean_text(x))

In [ ]:
# Let's see the max number of words in the tweets

df['n_words'] = df['text'].apply(lambda x:len(str(x).split()))
df['n_words'].max()

In [ ]:
# Achtung we have empty tweets

df['n_words'].min()

In [ ]:
sns.displot(df['n_words']);

# Create a Model

In [ ]:
import tensorflow as tf

In [ ]:
from sklearn import model_selection

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df['text'], df['sentiment'], test_size=0.30)

In [ ]:
X_train = np.array(X_train.values.tolist())
X_test = np.array(X_test.values.tolist())
y_train = np.array(y_train.values.tolist())
y_test = np.array(y_test.values.tolist())

# Binarizing labels

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train_enc = mlb.fit_transform([list(y_train)]) # pay attention to the []

#mlb.classes_

In [ ]:
y_train_enc, set(list(y_train))

# Tokening tweets

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()

tokenizer.fit_on_texts(X_train)

vocab_size = len(tokenizer.word_index) + 1
vocab_size

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)

In [ ]:
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
max([len(x) for x in X_train])

In [ ]:
maxlen = 100

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 20, input_length=maxlen),
    tf.keras.layers.GRU(units=32, dropout=0.2, recurrent_dropout=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

In [ ]:
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])